# 几种不同角度的Policy Gradient Expression的推导

- <font color=brown>符号说明：文中 $a_t和u_t$ 混用，都表示t时刻的action。</font>

## I. 直接推导目标和梯度

### I.1 目标

#### I.1.1 目标的两种展开方法

1. trajectory的期望rewards：
$$J(\theta)  = E_{\tau }R(\tau|\pi_{\theta}) = \sum_{\tau }P(\tau ;\theta )R(\tau )$$

2. 不同状态价值的均值：
$$\begin{align}
严格说应该是初始状态的价值期望值：\\
J(\theta) & = E_{s\sim d(s_0)}[V^{\pi_{\theta }}(s)]=\sum_{s_0\in \mathcal{S} }d(s_0)V^{\pi_{\theta }}(s_0)\\
也可以取s处于stationary\ distribution时的分布：\\
J(\theta) & = E_{s\sim d(s)}[V^{\pi_{\theta }}(s)]=\sum_{s\in \mathcal{S} }d(s)V^{\pi_{\theta }}(s)\\
\end{align}$$

#### I.1.2 目标展开式暗含的假设条件

- 期望值合理的前提是随机变量的分布稳定。也就是说，trajectory展开式中，假设了trajectory的分布稳定；状态价值展开式中，假设了状态s的分布稳定。<font color=red>但这两个条件在现实条件下都不成立。</font>
- 在状态价值展开式中，理论上，如果agent与环境交互的次数极大的条件下，$d(s)$可以用策略对应的stationary distribution。但是这个条件在真实环境中通常也很难满足。
- 尽管他们的分布处于不稳定状态，实践中还是会用monte carol抽样法直接估计期望的方式来得到估计量。

### I.2 用trajectory的期望rewards推导梯度

#### I.2.1 梯度的基本公式

$$\begin{align}
\nabla _{\theta }J(\theta ) 
& = \nabla _{\theta } \sum_{\tau }P(\tau ;\theta )R(\tau ) \\
& =  \sum_{\tau }\nabla _{\theta }P(\tau ;\theta )R(\tau ), {\color{green}{这步是因为\theta带给\tau的随机性都体现在P(\tau;\theta)里面}} \\
& =  \sum_{\tau }P(\tau ;\theta )\frac{\nabla _{\theta }P(\tau ;\theta )}{P(\tau ;\theta )}R(\tau ) \\
& =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )R(\tau ) 
\end{align}$$

$$\begin{align}
\nabla _{\theta }logP(\tau ;\theta ) 
& = \nabla _{\theta }log\left[ d(s_0)\prod_{t=0}^{H-1} P(s_{t+1}|s_t, u_t)*\pi_{\theta }(u_t|s_t) \right ]\\
& = \nabla _{\theta }\left[logd(s_0) + \sum_{t=0}^{H-1}logP(s_{t+1}|s_t, u_t) + \sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\right ]\\
& = \underset{0}{ \underbrace{\nabla _{\theta }logd(s_0)}}  + \underset{0}{\underbrace{\nabla _{\theta }\sum_{t=0}^{H-1}logP(s_{t+1}|s_t, u_t)}} + \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)\\
& = \nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t) \\
\end{align}$$

$$\begin{align}
取梯度：g & = \nabla _{\theta }J(\theta )\\
& =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )R(\tau ) \\
& = E_{\tau }\left[\nabla _{\theta }\sum_{t=0}^{H-1}log\pi_{\theta }(u_t|s_t)R(\tau )\right ]\\
& = E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t=0}^{H-1}r(s_t, u_t)\right ]
\end{align}$$

#### I.2.2 rewards to go

$$g = E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, u_{t'})\right ]$$

#### I.2.3 baseline

- 当b是常数时，下式成立：
$$\begin{align}
E_{\tau }[\nabla _{\theta }logP(\tau ;\theta )]b& = \nabla _{\theta } \sum_{\tau }P(\tau ;\theta )b \\
& = b\nabla _{\theta } \sum_{\tau }P(\tau ;\theta )\\
& = b\nabla _{\theta }1 = 0
\end{align}$$

- <font color=norange>**即使b是states的函数，只要b(s)不受action的影响，等式仍然成立。注意，这里关键是b(s)与action的关系，而不是b(s)与策略的关系。即使b(s)受策略的影响，比如图取值为状态价值函数V(s)也不改变结果。**</font>见下面分析过程：

$$\begin{align}
E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)b(s_t)\right ]
& = E_{\tau }\left[ \sum_{t=0}^{H-1}[\nabla _{\theta }log\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ]\\
& = \underset{s_t\in\mathcal{S},a_t\in \mathcal{A}}{E}\left[ \sum_{t=0}^{H-1}[\nabla _{\theta }log\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)\sum_{a_t\in \mathcal{A}} [\pi_{\theta }(a_t|s_t)\nabla _{\theta }log\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)\sum_{a_t\in \mathcal{A}}[\nabla _{\theta }\pi_{\theta }(a_t|s_t)]V^{\pi }(s_t)\right ],{\color{green}{因为V(s_t)与a_t无关}}\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)V^{\pi }(s_t)\sum_{a_t\in \mathcal{A}}\nabla _{\theta }\pi_{\theta }(a_t|s_t)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)V^{\pi }(s_t)\nabla _{\theta }\left(\sum_{a_t\in \mathcal{A}}\pi_{\theta }(a_t|s_t)\right)\right ]\\
& = \sum_{t=0}^{H-1}\left[\sum_{s_t\in\mathcal{S}}P(s_t)V^{\pi }(s_t)\nabla _{\theta }1\right ]=0
\end{align}$$

- 为了降低估计量的方差，b(s)的sub-optimal取值为$b(s)=V^{\pi}(s)$。
  - 将action advantage记为$A_t=Q^{\pi}(s_t,u_t)-V^{\pi}(s_t)$
  - 因为 $Q^{\pi}(s_t,u_t) = \sum_{t'=t}^{H-1}E_{\pi}[r(s_{t'}, u_{t'})|s_t,u_t]$
$$\begin{align}
g & =  E_{\tau }\nabla _{\theta }logP(\tau ;\theta )[R(\tau ) - b]\\
&= E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( \sum_{t'=t}^{H-1}r(s_{t'}, u_{t'})-b(s_t)\right)\right ]\\
&= E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)\left( Q(s_{t}, u_{t})-V(s_t)\right)\right ]\\
&= E_{\tau }\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t|s_t)A_t\right ]\\
\end{align}$$

- 此时，策略梯度的估计量为：
$$\begin{align}
\hat g & = \frac{1}{N} \sum _{i=1 }^N\nabla _{\theta }logP(\tau_i;\theta )[R(\tau_i) - b]\\
&= \frac{1}{N} \sum _{i=1 }^N\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\left( \sum_{t'=t}^{H-1}r(s_{t'}^{(i)}, u_{t'}^{(i)})-V^{\pi}(s_t^{(i)})\right)\right ]\\
&= \frac{1}{N} \sum _{i=1 }^N\left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(u_t^{(i)}|s_t^{(i)})\hat A_t\right ]
\end{align}$$

- <font color=blue>**不同估计方法的bias-varince tradeoff**</font>：
  1. 如果用Monte Carol估$R(\tau_i) - V(s_t^{(i)})$，得到的估计量unbiased，但是varince很大
  2. 改用DNN先估 $V(s_t)或者Q(s_t)$，然后得到Advantage function的估计量的方法能降低variance，但是此时不再满足unbiased条件。

### I.3 <font color=gray>用state function期望值推导梯度</font>

- <font color=orange>这种推导方式的假设比用trajectory的假设多了一条：d(s)不受策略影响。而这一点在现实中通常不成立，所以这种方式描述现实的能力不如用trajectory期望值的推导方式。</font>

- <font color=red>**假如d(s)不受策略影响**</font>：这个假设通常不成立，但是能简化策略梯度的推导
$$\begin{align}
g = \nabla _{\theta }J(\theta) & = \nabla _{\theta }E_{s\sim d(s)}[V^{\pi_{\theta }}(s)] \\
& = \nabla _{\theta }\sum_{s\in \mathcal{S} }d(s)V^{\pi_{\theta }}(s) \\
& = \sum_{s\in \mathcal{S} }d(s)\nabla _{\theta }V^{\pi_{\theta }}(s) \\
\end{align}$$

- 将状态价值函数的梯度展开可以得到一个线性方程组：
$$\begin{align}
\nabla _{\theta }V^{\pi_{\theta }}(s)
& = \nabla _{\theta }\left[ \sum_{a\in \mathcal{A}} \pi(a|s)Q^{\pi }(s,a)\right]\\
& = \sum_{a\in \mathcal{A}} \left[ Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \pi(a|s)\nabla _{\theta } Q^{\pi }(s,a)\right]\\
& = \sum_{a\in \mathcal{A}} \left[ Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \pi(a|s)\nabla _{\theta } \left (r(s,a,s') + \gamma \sum_{s'\in \mathcal{S}}P(s'|s,a) V^{\pi }(s')\right )\right]\\
& = \sum_{a\in \mathcal{A}} \left[ Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \pi(a|s) \gamma \sum_{s'\in \mathcal{S}}P(s'|s,a)\nabla _{\theta }V^{\pi }(s')\right]\\
& = \sum_{a\in \mathcal{A}}  Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \gamma \sum_{a\in \mathcal{A}}\pi(a|s) \sum_{s'\in \mathcal{S}}P(s'|s,a)\nabla _{\theta }V^{\pi }(s')\\
& = \sum_{a\in \mathcal{A}}  Q^{\pi }(s,a)\nabla _{\theta } \pi(a|s) + \gamma \sum_{s'\in \mathcal{S}}P^{\pi}(s'|s)\nabla _{\theta }V^{\pi }(s')\\
\end{align}$$

- 求解该线性方程组的解为：<font color=brown>[详见math of RL lemma 9.2]</font>
$$\begin{align}
\nabla _{\theta }V^{\pi_{\theta }}(s)
& = \sum_{s'\in \mathcal{S}}Pr^{\pi}(s'|s)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\\
其中：\\
Pr^{\pi}(s'|s) & =\sum_{k=0}^{\infty}\gamma^k[P^{\pi,k }(s'|s)],\ P^{\pi,k }(s'|s)是s经k步转向s'的概率
\end{align}$$

- 代入策略梯度式：
$$\begin{align}
g & = \sum_{s\in \mathcal{S} }d(s)\nabla _{\theta }V^{\pi_{\theta }}(s) \\
& = \sum_{s\in \mathcal{S} }d(s)\sum_{s'\in \mathcal{S}}Pr^{\pi}(s'|s)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\\
& = \sum_{s'\in \mathcal{S}}\left(\sum_{s\in \mathcal{S} }d(s)Pr^{\pi}(s'|s)\right)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\\
& = \sum_{s'\in \mathcal{S}}\rho (s')\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s')Q(s',a)\ ,\ 更换符号\\
& = \sum_{s\in \mathcal{S}}\rho (s)\sum_{a\in\mathcal{A}}\nabla_{\theta}\pi(a|s)Q(s,a) \\
& = \sum_{s\in \mathcal{S}}\rho (s)\sum_{a\in\mathcal{A}}\pi_{\theta}(a|s)\nabla_{\theta}log\pi_{\theta}(a|s)Q^{\pi}(s,a) \\
& = \underset{s\sim \rho^{\pi}, a\sim \pi}{E}\nabla_{\theta}log\pi_{\theta }(a|s)Q^{\pi }(s,a) \\
\end{align}$$

## II. Importance sampling形式的目标和梯度

### II.1 <font color=gray>基于trajectory的期望rewards的Importance sampling</font>

- <font color=red>这种方式推导出来的式子中有累乘，导致直接用它抽样的话，方差极大。所以这里的推导只做分析参考，不会在实践中使用。</font>

#### II.1.1 目标$J(\theta)$的Importance sampling

$$\begin{align}
J (\theta )& = \underset{\tau\sim \pi _\theta}{E} [R(\tau)] = \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  [\frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}R(\tau)] \\
其中：\\
\frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)} & = \frac{P(s_0)\prod_{t=0}^{H-1}\pi_{\theta }(a_t|s_t)P(s_{t+1}|s_t, a_t)}{P(s_0)\prod_{t=0}^{H-1}\pi_{\bar \theta }(a_t|s_t)P(s_{t+1}|s_t, a_t)} 
= \frac{\prod_{t=0}^{H-1}\pi_{\theta }(a_t|s_t)}{\prod_{t=0}^{H-1}\pi_{\bar \theta }(a_t|s_t)} 
= \prod_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar \theta }(a_t|s_t)}  \\
代入上式：\\
J(\theta) &= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  [\frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}R(\tau)]= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  [ \prod_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar \theta }(a_t|s_t)} R(\tau)]
\end{align}$$

#### II.1.2 梯度$\nabla_\theta J(\theta)$的Importance sampling

$$\begin{align}
\nabla _{\theta }J(\theta) & =  \underset{\tau\sim \pi_{\theta}}{E} \ \  \nabla _{\theta }logP(\tau ;\theta )R(\tau ) \\
& = \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}\nabla _{\theta }logP_{\theta}(\tau)R(\tau )  \\
& = \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \frac{P_{\theta}(\tau)}{P_{\bar \theta}(\tau)}\nabla _{\theta }log\pi _{\theta}(\tau)R(\tau )  \\
&= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \left [\left( \prod_{t=0}^{H-1}\frac{\pi_{\theta,t }}{\pi_{\bar \theta, t }} \right)\left(\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta,t }\right)\left(\sum_{t=0}^{H-1}r_t\right)\right]\\
&= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \left [\left( \prod_{t^{''}=0}^{t}\frac{\pi_{\theta,t^{''} }}{\pi_{\bar \theta,t^{''} }} \right)\left( \prod_{t^{'''}={t+1
}}^{H-1}\frac{\pi_{\theta, t^{'''} }}{\pi_{\bar \theta, t^{'''} }} \right)\left(\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta,t }\sum_{t'=t}^{H-1}r_t\right)\right]\\
&= \underset{\tau\sim \pi_{\bar \theta}}{E} \ \  \left [\sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta,t }(\prod_{t^{''}=0}^{t}\frac{\pi_{\theta ,t^{''} }}{\pi_{\bar \theta ,t^{''}}})(\sum_{t'=t}^{H-1}r_t)\right]
\end{align}$$

- 实践中不用这个式子，因为上式中：
$$\prod_{t^{''}=0}^{t}\frac{\pi_{\theta ,t^{''} }}{\pi_{\bar \theta ,t^{''}}} \rightarrow exponential\ in\  T，方差极大$$

### II.2 IS梯度的近似和它对应的surrogate loss目标

- <font color=blue>Natural PG、TRPO和PPO的梯度形式和这里用importance sampling推导的近似表达式非常像。但要注意：</font>
  1. Natural PG推导它的方式并不是从importance sampling的角度。<font color=brown>[详见Natural PG部分的notes]</font>
  2. 在原Natural PG论文中没有使用IS形态的表达式，在TRPO论文中使用了类似IS的表述形式，但仅仅是为了更清晰地描述新旧策略在算法中的角色。<font color=green>**并没有真的使用IS方法抽样数据。**</font><font color=brown>[具体可参考PPO论文part 5公式(13)-公式(14)]</font>
  3. 这里就是用原梯度的importance sampling形式推导该近似表达式，实际得到的表达式和上述算法中实际使用的表达式有一点差异。<font color=red>这里每个trajectory对应的效用$Q(s_t,a_t)$是新策略条件下的效用，而上述算法中使用的梯度表达式中，trajectory的效用是旧策略条件下得到的。</font>

#### II.2.1 用state-action的的边际分布得到Importance sampling梯度的近似

- 另一种推导方式：假设$(s_t, a_t)\sim P_{\theta}(s_t, a_t)$是state-action的marginal distribution。
- 这种形式的优点是没有累乘，所以没有前一种推导方式中方差太大的问题。代价是得到的是策略梯度的近似估计量。实践中可用的原因是，尽管是近似估计量，但它导致的估计误差有明确的上限。<font color=brown>[详见后文]</font>

- 推导式：
$$\begin{align}
g & = \underset{\tau \sim \pi_{\theta }}{E} \left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\right ]\\
 & = \underset{\tau \sim \pi_{\theta }}{E} \left[ \sum_{t=0}^{H-1}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\right ]\\
& = \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\theta }(a_t, s_t)}{E} \nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\\
& = \sum_{t=0}^{H-1}\sum_{a_t\in\mathcal{A},s_t\in\mathcal{S}} P_{\theta }(a_t,s_t)\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\\
& = \sum_{t=0}^{H-1}\sum_{a_t\in\mathcal{A},s_t\in\mathcal{S}} P_{\bar \theta }(a_t,s_t)\frac{P_{\theta }(a_t,s_t)}{P_{\bar \theta }(a_t,s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)\\
& = \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)}{E}\frac{P_{\theta }(a_t,s_t)}{P_{\bar \theta }(a_t,s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t) \\
& = \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)}{E}\frac{P_{\theta }(s_t)\pi_{\theta }(a_t|s_t)}{P_{\bar \theta }(s_t)\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t) \\
& \approx \sum_{t=0}^{H-1}\underset {(a_t, s_t)\sim P_{\bar \theta }(a_t, s_t)}{E}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t) \\
& = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})
\end{align}$$

- 上面推导过程的说明：
  - 第二个等式成立是因为：将$\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})$替换为$Q^{\pi_{\theta }}(s_t,a_t)$在trajectory的期望条件下是成立的。
  - 第三个等式成立是因为：$\nabla _{\theta }log\pi_{\theta }(a_t|s_t)Q^{\pi_{\theta }}(s_t,a_t)$ 是$(s_t,a_t)$为变量的函数。
    1. $\pi_{\theta }(a_t|s_t)$ 是$(s_t,a_t)$为变量的函数
    2. $Q^{\pi_{\theta }}(s_t,a_t)$的展开式中虽然有$t'>t$的所有time-steps的影响，但是所有的$a_{t'},s_{t'}$的不确定性都被期望积掉了。
  - 约等式中直接去掉了$\frac{P_{\theta}(s_t)}{P_{\bar \theta}(s_t)}$，这是导致最终近似估计式简化的原因，也是导致估计量近似而不是等式的原因。

#### II.2.2 surrogate loss目标

- 取梯度为IS梯度近似表达式时，为了利用现成的深度学习框架，可以用它反推一个surrogate loss

$$\begin{align}
当梯度取下式时：\\
g & = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\nabla _{\theta }log\pi_{\theta }(a_t|s_t)\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
对应的目标可以用：\\
L^{IS}(\theta ) & = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
对L(\theta)求梯度很容易得到：\\
& \nabla_{\theta }L^{IS}(\theta ) = g
\end{align}$$

- 用梯度反推得到的surrogate loss可以从importance sampling角度来理解其含义：
  - 严格地说，这里的surrogate loss不算真正的importance sampling loss，因为trajectory的rewards如果要取importance sampling的期望值的话，应该用$(s_t, a_t)$的边际分布来计算importance weight，但这里用的是$\pi(a_t|s_t)$，此外还有r采样在实践中应该来自旧策略，而IS中的推导是新策略这点差异。
  - 但一般会将这个loss用importance sampling来标记，这是因为它是真正的importance sampling loss的近似值，并且很多地方都没有区分r采样的所使用的策略差异，直接把IS中的reward改成旧策略的采样，然后标记为IS loss。
$$\begin{align}
L^{IS}(\theta ) & = \underset {\tau \sim \pi_{\bar\theta }}{E}\sum_{t=0}^{H-1}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
& = \sum_{t=0}^{H-1}E_{s_t\sim P_{\bar \theta }(s_t)}E_{a_t\sim {\pi_{\bar \theta }}}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}\sum_{t'=t}^{H-1}r(s_{t'}, a_{t'})\\
PPO等算法实际上用的是：\\
& = \sum_{t=0}^{H-1}E_{s_t\sim P_{\bar \theta }(s_t)}E_{a_t\sim {\pi_{\bar \theta }}}\frac{\pi_{\theta }(a_t|s_t)}{\pi_{\bar\theta }(a_t|s_t)}{\color{red}{Q^{\pi_{\bar \theta}}}}(s_t,a_t)\\
\end{align}$$

## III. Importance Sampling要点 

- 思路：如果没办法直接从 $p(x)$ 中采样数据，可以从 $q(x)$ 中采样数据，$x^i\sim q(x)$
$$
\begin{split}
E_{x\sim p}[f(x)]&=\int f(x)p(x)dx=\int f(x)\underbrace{\frac{p(x)}{q(x)}}_{\text{importance weight}}q(x)dx\\
&=E_{x\sim q}[f(x)\frac{p(x)}{q(x)}]
\end{split}
$$

- 期望相同
$$E_{x\sim p}[f(x)]=E_{x\sim q}[f(x)\frac{p(x)}{q(x)}]$$

- 方差不同： q的分布与p的分布越接近，方差越小。如果两个分布差异很大，则估计的结果方差大，也就是说大多数时候估计得到的结果偏差很大。
$$\begin{split}
    \text{Var}_{x\sim q}[f(x)\frac{p(x)}{q(x)}]&=E_{x\sim q}[(f(x)\frac{p(x)}{q(x)})^2]-(E_{x\sim q}[f(x)\frac{p(x)}{q(x)}])^2\\
    &=E_{x\sim p}[f^2(x)\frac{p(x)}{q(x)}] - (E_{x\sim p}[f(x)])^2
\end{split}$$